In [214]:
#include("Hamiltonian-maker.jl")
using DMRJtensor

spinmag = 1.0;
Sx,Sy,Sz,Sp,Sm,O,Id = spinOps(spinmag);

In [244]:
function S_pow_iterative(H, spin_mag::Float64, curr_op, start_index::Integer, npowers::Integer, curr_power::Integer, J)

    Sop_arr = [Sx,Sy,Sz] # spin operators
    nSpin = Int(2*spin_mag + 1) # dimension of spin space
    nterms = size(H,1) # total size of the MPO's onsite H
    
    if (curr_power == npowers) # if at bottom of iterative tree
                
        for dim = 1:3 # loop over (x,y,z)
            
            idx_start = start_index + nSpin*(dim-1) # starting point along first column / last row
            lr_start = nterms - nSpin + 1 # index location of last row

            op_h = curr_op*Sop_arr[dim]; # generate final operator
            
            H[idx_start:(idx_start+nSpin-1),1:nSpin] = J*op_h # first column (has J!)
                
            H[lr_start:(lr_start+nSpin-1),idx_start:(idx_start+nSpin-1)] = op_h # last column (no J!)
                
        end
    else # otherwise, keep iterating
        for dim = 1:3 # loop over (x,y,z)
            op_h = curr_op*Sop_arr[dim] # add term to operator product
            stride_idx = 3^(npowers-curr_power) # figure out how wide the next block will be
            # iterate!
            S_pow_iterative(H, spin_mag, op_h, start_index + nSpin*stride_idx*(dim-1), npowers, curr_power+1, J) 
        end
    end
        
end

function H_SU2(s::Float64, J_arr)
                        
    n = Int(2*s + 1) # dimension of spin space
    max_p = n-1 # max allowed power of (S*S)^p terms
    nterms = n*2 # keep track of Id and O terms in MPO
    for p = 1:max_p # find size over all allowed powers
        nterms = nterms + n*(3^p)
    end
    H_n = zeros(ComplexF64, nterms, nterms) # initialize
                     
    # set first column and last row identities
    H_n[1:n,1:n] = Id
    H_n[(nterms-n+1):nterms,(nterms-n+1):nterms] = Id
                        
    # we start in H just after the first [n x n] row block (or column block, if on last row)
    start_index = 1 + n;
    
    for p = 1:max_p # add terms associated with each power, (S*S)^p
        S_pow_iterative(H_n, s, Id, start_index, p, 1, J_arr[p]) # iterative construction of (S*S)^p
        start_index = start_index + n*(3^p) # keep track of the H index for each (S*S)^p group!
    end
    return H_n 
end

Ns = 5 # number of sites
J_arr = [1.0, 2.0]; # J_p array, e.g. H = sum_p[ J_p (S*S)^p ]
H_onesite = H_SU2(spinmag, J_arr) # make the onsite term
H_mpo = makeMPO(H_onesite,size(Id,1),Ns); # make the MPO!


In [248]:
println(H_mpo[3])
mpo_dims = H_mpo[3].size
mpo_mat = H_mpo[3].T


println(mpo_dims) # has indices [a1,s1,s2,a2]
for a1 = 1:mpo_dims[1] # major row
    for s1= 1:mpo_dims[2] # minor row
        for a2 = 1#:mpo_dims[4] # major col
            for s2=1:mpo_dims[3] # minor col
                idx_h = a1 + (s1-1)*mpo_dims[1] + (s2-1)*mpo_dims[1]*mpo_dims[2] + (a2-1)*mpo_dims[1]*mpo_dims[2]*mpo_dims[3]

                print(round(abs(mpo_mat[idx_h]),digits=1))
                print(" ")
            end
            print("| ")
        end
        print("\n")
    end
    for row_sep = 1:(mpo_dims[3])#*mpo_dims[4])
        print("----")
    end
    print("\n")
end

#println(Sx)

printing regular tensor of type: tens{Complex{Float64}}
size = (14, 3, 3, 14)
T = Complex{Float64}[1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 1.0 + 0.0im]...

(14, 3, 3, 14)
1.0 0.0 0.0 | 
0.0 1.0 0.0 | 
0.0 0.0 1.0 | 
------------
0.0 0.7 0.0 | 
0.7 0.0 0.7 | 
0.0 0.7 0.0 | 
------------
0.0 0.7 0.0 | 
0.7 0.0 0.7 | 
0.0 0.7 0.0 | 
------------
1.0 0.0 0.0 | 
0.0 0.0 0.0 | 
0.0 0.0 1.0 | 
------------
1.0 0.0 1.0 | 
0.0 2.0 0.0 | 
1.0 0.0 1.0 | 
------------
1.0 0.0 1.0 | 
0.0 0.0 0.0 | 
1.0 0.0 1.0 | 
------------
0.0 0.0 0.0 | 
1.4 0.0 1.4 | 
0.0 0.0 0.0 | 
------------
1.0 0.0 1.0 | 
0.0 0.0 0.0 | 
1.0 0.0 1.0 | 
------------
1.0 0.0 1.0 | 
0.0 2.0 0.0 | 
1.0 0.0 1.0 | 
------------
0.0 0.0 0.0 | 
1.4 0.0 1.4 | 
0.0 0.0 0.0 | 
------------
0.0 1.4 0.0 | 
0.0 0.0 0.0 | 
0.0 1.4 0.0 | 
------------
0.0 1.4 0.0 | 
0.0 0.0 0.0 | 
0.0 1.4 0.0 | 
------------
2.0 0.0 0.0 | 
0.0 0.0 0.0 | 
0.0 0.0 2.0 | 
------------
0.0 0.0 0.0 | 
0.0 0.0 0.0 | 
0.0 0.0 0.0 | 
------------
